<a href="https://colab.research.google.com/github/mmorales20/CS470/blob/master/11_11_DataConverter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

>[Packages](#scrollTo=tkKZZHqhNzuN)

>[Reading in CSV Files](#scrollTo=q7zSBFfcPXG_)

>[Categorical Data finder](#scrollTo=LX7CdYrqTwe8)

>[Numeric Data Finder](#scrollTo=5yOTozz4UfvX)

>[SDT Changes](#scrollTo=BeheiUGPVn75)

>[ETC](#scrollTo=wSVYcrMuWl-U)

>[Runner](#scrollTo=zc5tcEDYWyw5)



# Packages


In [0]:
#Download packages
import pandas as pd
import re
import xml.etree.ElementTree as ET

#Download package for File Explorer pop up
from tkinter import filedialog
from tkinter import *

import os

# Reading in CSV Files

In [0]:
#Display a file explorer and allow user to select a CSV file that contains the data
def getFile(sdtFile):
    root = Tk()
    root.filename =  filedialog.askopenfilename(initialdir = "/",title = "Select file",filetypes = (("text files","*.csv"),("all files","*.*")))
    #Since the above line goes off the screen this is the entire line:
    #root.filename =  filedialog.askopenfilename(initialdir = "/",
          #title = "Select file",filetypes = 
          #(("text files","*.csv"),("all files","*.*")))
    
    changePath(root.filename,sdtFile)
    root.destroy()
    return(root.filename)

In [0]:
#Change the path based on the file selected by the user
def changePath(path,sdtFile):
    tree = ET.parse(sdtFile)
    root = tree.getroot() 
    
    for elem in root.findall('.//Datasources/Datasource/Host'):
        elem.text=path
    
    for elem in root.findall('.//Datasources/Datasource/Name'):
        elem.text=path
        
    tree.write(sdtFile,encoding="utf-8", xml_declaration=True)

In [0]:
#Read in each column of the CSV file that was selected
def readCSV(csvName):
    df = pd.read_csv(csvName)
    print("reading csv done")
    return df

# How to Make a .exe 

Download Python from the web, and select "Add Python 3.7 to PATH"

File -> Download As -> Python (.py)

Open Windows Command Prompt (either look up Windows Command Prompt in Search bar or Windows button + R, 
then type in "cmd")

In Command Prompt - 

> pip install pyinstaller


> cd yourfilepathhere


> pyinstaller --onefile nameofyourfile.py

Then check in the File explorer that were you had your original .py file, and there should be a folder names "dist"

The executable file should be in there



# Categorical Data Finder

In [0]:
#Add each categoircal data column to a dataframe
def getCat(df,colorName):
    x=0
    for name in df.columns:
        if df.dtypes[x] == 'object':
             if len(df[name].astype('category').cat.categories.tolist()) >1 and len(df[name].astype('category').cat.categories.tolist()) <3 and colorName=="": 
             #View Full Code:
             #if len(df[name].astype('category').cat.categories.tolist()) >1 
                  #and len(df[name].astype('category').cat.categories.tolist()) 
                  #<101 and colorName=="": 
                colorName=df.columns[x]
        x= x+1
    return colorName

In [0]:
#Within each column save each unique attribute to a list
def getCatCol(df,colorName):
    return df[colorName].astype('category').cat.categories.tolist()

# Numeric Data Finder

In [0]:
#Find numeric (int and float) data and add the column names to a list
def typeDefiner(df,catFound,sdtFile):
    colList = df.columns
    colList
    x = 0
    colNames = []
    for name in colList:
        if df.dtypes[x] == 'object':# or df.dtypes[x] == 'int64':
            pass
        else:
            colNames.append(colList[x])    
        x = x + 1
    
    #Finds number of numeric inputs in case of no categorical data
    file = ET.parse(sdtFile)
    root = file.getroot()
    inputCount=0
    for elem in root.findall('.//InputFields'):
        for inputs in elem.findall('.//InputField'):
            inputCount+=1
    
    #Resets part of color section and input if there is no option for color
    if not catFound and inputCount>len(colNames):
        resetToDefault(sdtFile)
    print("input count is "+str(catFound) +str(inputCount))
    print("objects removed")
    print(colNames)
    return colNames

# Random nameID Generator

In [0]:
# This is to generate random name IDs for one of our subelements
def randomNameIDGenerator():
    name = "~"
    z = 0
    while z < 10:
        value = randint(0, 10)
        name += str(value)
        z = z + 1
    return name  

# SDT Changes

In [9]:
#Fully functional - 
#Clears out the tag in the color section and formats the SDT file
def toNumeric(sdtFile):

    #Finds keyValuePairs tag and clears out the tag.
    file = ET.parse(sdtFile)
    root = file.getroot()
    for elem in root.findall('.//Glyph/Color/RGB/Function[@type="Text Field To Value"]/KeyValuePairs'):
    #View Full Line:
    #for elem in root.findall('.//Glyph/Color/RGB/Function[@type="Text Field 
          #To Value"]/KeyValuePairs'):
        elem.clear()

    #Finds end tag and removes end tag from code
    for parent in root.findall('.//KeyValuePairs/..'):
        for element in parent.findall('KeyValuePairs'):
            parent.remove(element)
   #Sets up function to use numeric data with the appropriate lines
    for elem in root.findall('.//Color/RGB/..'):
        function_elem = elem.find('.//Function')
        function_elem.set("type","Linear Interpolation")
        attrib = {'type': 'BoundField'}
        minmax = ET.SubElement(check_req_elems, 'MinMax', attrib)
        minmax.text= "\n\t\t\t\t\t\t\t"
        min = ET.SubElement(minmax, 'Min')
        min.text = '0'
        min.tail="\n\t\t\t\t\t\t"
        minmax.tail="\n\t\t\t\t\t"
    file.write(sdtFile)
    print("color fixed")

SyntaxError: ignored

In [0]:
#Accepted Params: list of column names, the .SDT file, column name, and boolean
def XMLParser(colNames,sdtFile,colorName,catFound):
    print(colNames)

    tree = ET.parse(sdtFile)
    root = tree.getroot()
    count=0;
    print(len(colNames))
    for elem in root.iter('InputField'):
        print(count)
        if elem.get('name')!="color":
            elem.set('field', colNames[count])
            count+=1
        elif not catFound:
            elem.set('field', colNames[count])
            count+=1
        else:
            elem.set('field', colorName)
        
    tree.write(sdtFile)
    print("XML updated")

In [0]:
#
def setupColor(colorCol,sdtFile):
    file = ET.parse(sdtFile)
    root = file.getroot()
    x=0
    if(colorCol==None):#Changes Color code to take in numeric data
        toNumeric(sdtFile)
        
    else:
      #sets up prexisting key value pairs with the first 2 categorical options
        for parent in root.findall('.//Color/RGB/Function[@type="Text Field To Value"]/KeyValuePairs'):
        #View Full Line:
        #for parent in root.findall('.//Color/RGB/Function[@type="Text Field 
              #To Value"]/KeyValuePairs'):
            for elem in parent.findall('KeyValuePair/Key'):
                elem.text=colorCol[x]
                x=x+1

        #Creates new key value pairs for the rest of the categorical list
            if(len(colorCol)>2):
                while x<len(colorCol):
                    for elem in root.findall('.//Color/RGB/Function[@type="Text Field To Value"]'):
                    #View Full Line:
                    #for elem in root.findall('.//Color/RGB/
                            #Function[@type="Text Field To Value"]'):    
                        pairs = elem.find('.//KeyValuePairs')
                        #fixes spacing so new pair will be at right 
                          #column/indent area
                        #Adds key value pair as subelement to key value pairs
                        for child in pairs.findall('.//KeyValuePair'):
                            child.tail="\n\t\t\t\t\t\t\t"
                        keyVal =ET.SubElement(pairs, 'KeyValuePair')
                        keyVal.text="\n\t\t\t\t\t\t\t\t"
                        keyVal.tail="\n\t\t\t\t\t\t"
                        keyValP= ET.SubElement(keyVal, 'Key')
                        keyValP.text=colorCol[x]
                        keyValP.tail="\n\t\t\t\t\t\t\t\t"
                        key = ET.SubElement(keyVal, 'Value')
                    
                        key.tail="\n\t\t\t\t\t\t\t"
                        x=x+1
    
                        x=0

                        #Creates an array of random colors for the categorical data
                        count=0
                        colorList=[] 
                        while count<len(colorCol):
                            randomCol(colorList)
                            count=count+1

                        
                        #sets up color values for each key value pair
                        for elem in root.findall('.//Color/RGB/Function[@type="Text Field To Value"]/KeyValuePairs'):
                        #View Full Line:
                        #for elem in root.findall('.//Color/RGB/Function[@type=
                              #"Text Field To Value"]/KeyValuePairs'):
                            for element in elem.findall('KeyValuePair/Value'):
                                element.text=colorList[x]
                                print(colorList[x])
                                x=x+1
        file.write(sdtFile,encoding="utf-8", xml_declaration=True)


In [0]:
#
def moveToNewFile(sdtFile):
    file = ET.parse(sdtFile)
    base=os.path.basename(csv)
    newFile =os.path.splitext(base)[0]+".sdt"
    file.write(newFile,encoding="utf-8", xml_declaration=True)
    
    return newFile

In [0]:
def resetToDefault(sdtFile):
    file = ET.parse(sdtFile)
    root = file.getroot()
    print("resetting")
    removeElement=None
    for parent in root.findall('.//Glyph/Color/RGB'):
        for elem in parent:
            if(removeElement==None):
                removeElement = parent.find('Binding')
        if(removeElement!=None):
            parent.remove(removeElement)
            removeElement=None
    
    for parent in root.findall('.//InputFields'):
        for elem in parent:
            if(removeElement==None):
                removeElement =  parent.find('InputField[@name="color"]')
              
        if (removeElement!=None):
            parent.remove(removeElement)
            reMoveElement=None
    
    count=0
    #fixes indent spacing for new last element
    for parent in root.findall('.//InputFields'):
        for elem in parent:
            if(count==len(parent)-1):
                elem.tail="\n\t"
            count+=1
        
    file.write(sdtFile,encoding="utf-8", xml_declaration=True)

# Random Color Generator


In [0]:
#Random Color Generator
def randomColors(colorList):
    import random
    rVal = random.random()*255
    rVal=int(rVal)
    gVal = random.random()*255
    gVal=int(gVal)
    bVal = random.random()*255
    bVal=int(bVal)
    color=str(rVal)+","+str(gVal)+","+str(bVal)
    colorList.append(color)

#Runner

In [0]:
def runAll(sdtFile):
    csv = getFile(sdtFile)
    df = readCSV(csv)
    colorName=getCat(df,colorName)
    if(colorName!=""):
        catFound=True
        colorCol=getCatCol(df,colorName)
  
    sdtFile=moveToNewFile(sdtFile,csv)
    colNames = typeDefiner(df,catFound,sdtFile)
    XMLParser(colNames, sdtFile,colorName,catFound)
    setupColor(colorCol, sdtFile)

In [19]:
colorCol= None
colorName=""
catFound=False
runAll('emptySDT.sdt')

TclError: ignored